In [1]:
var csv = require('csv-parser');  

In [2]:
var fs = require('fs');

In [3]:
var csv_train_file_path = './data/train.csv'

In [9]:
// var X = []
var toxic_y = []
var severe_toxic_y = []

fs.createReadStream(csv_train_file_path)  
  .pipe(csv())
  .on('data', (row) => {
    X.push(row.comment_text);
    toxic_y.push(row.toxic)
    severe_toxic_y.push(row.severe_toxic)
  })
  .on('end', () => {
    console.log('CSV file successfully processed');
  });

CsvParser {
  _readableState:
   ReadableState {
     objectMode: true,
     highWaterMark: 16,
     buffer: BufferList { head: null, tail: null, length: 0 },
     length: 0,
     pipes: null,
     pipesCount: 0,
     flowing: true,
     ended: false,
     endEmitted: false,
     reading: false,
     sync: false,
     needReadable: false,
     emittedReadable: false,
     readableListening: false,
     resumeScheduled: true,
     paused: false,
     emitClose: true,
     autoDestroy: false,
     destroyed: false,
     defaultEncoding: 'utf8',
     awaitDrain: 0,
     readingMore: false,
     decoder: null,
     encoding: null },
  readable: true,
  _events:
   [Object: null prototype] {
     prefinish: [Function: prefinish],
     unpipe: [Function: onunpipe],
     drain: [Function: pipeOnDrainFunctionResult],
     error: [Function: onerror],
     close:
      { [Function: bound onceWrapper] listener: [Function: onclose] },
     finish:
      { [Function: bound onceWrapper] listener: [F

CSV file successfully processed


In [5]:
// var nlp = require('compromise') // TODO normalize text strings if needed

CSV file successfully processed


In [10]:
var toxic_labels = toxic_y.map(Number);
var severe_toxic_labels = severe_toxic_y.map(Number);

var labels = [];
for(var i = 0; i , i < toxic_y.length; i++) {
    if(toxic_labels[i] == 1 || severe_toxic_labels[i] == 1) {
        labels.push(1);
    } else {
        labels.push(0);
    }
}

159571

In [6]:
// TODO try normalize if needed, skip now

// function preprocess(text) {
//    return nlp(text).normalize({case:false, puctuation:false, plurals:true, parentheses:true, possessives:true, honorifics:true, verbs:true}).out('text');
// }

In [7]:
// var X_preprocessed = X.map(preprocess);

In [8]:
// fs.writeFileSync('./data/X_preprocessed.json', JSON.stringify(X_preprocessed) , 'utf-8'); 

In [11]:
var X = JSON.parse(fs.readFileSync('./data/X_preprocessed.json'));

In [8]:
X.length == labels.length

true

In [12]:
function cleanString(input) {
    var output = "";
    for (var i=0; i<input.length; i++) {
        if (input.charCodeAt(i) <= 127) {
            output += input.charAt(i);
        }
    }
    return output;
}

In [13]:
var X_c = X.map(cleanString);

In [14]:
String.prototype.isEmpty = function() {
    return (this.length === 0 || !this.trim());
};

[Function]

In [15]:
var removal_indices = []
var X = X.filter(function(text, i) {
    var empty = text.isEmpty();
    if (empty == true){
        removal_indices.push(i);
    }
    return !empty;
});

In [16]:
removal_indices.reverse().forEach(function(i) {
    labels.splice(i, 1);
    X_c.splice(i, 1);
});

In [17]:
 labels.length == X.length && X.length == X_c.length

true

### Bag-of-words

In [18]:
var toxic_texsts = [];
var non_toxic_texsts = [];
for(var i = 0; i < labels.length; i++) {
    if (labels[i] == 1) {
        toxic_texsts.push(X_c[i]);
    } else {
        non_toxic_texsts.push(X_c[i]);
    }
}

143807

In [19]:
var toxic_texsts = toxic_texsts.join(' ');
var non_toxic_texsts = non_toxic_texsts.join(' ');

In [15]:
// taken and modified from https://github.com/techfort/mimir

In [20]:
function nGram(n) {
  if (typeof n !== 'number' || isNaN(n) || n < 1 || n === Infinity) {
    throw new Error('`' + n + '` is not a valid argument for n-gram')
  }

  return grams

  // Create n-grams from a given value.
  function grams(value) {
    var nGrams = []
    var index

    if (value === null || value === undefined) {
      return nGrams
    }

    value = value.slice ? value : String(value)
    index = value.length - n + 1

    if (index < 1) {
      return nGrams
    }

    while (index--) {
      nGrams[index] = value.slice(index, index + n)
    }

    return nGrams
  }
}

function extractDictionary(textArray, f) {
    var dict = {},
      keys = [],
      words;
    
    textArray = Array.isArray(textArray) ? textArray : [textArray];
    textArray.forEach(function (text) {
      words = f(text);
      words.forEach(function (word) {
        word = word.toLowerCase();
        if (!dict[word] && word !== '') {
          dict[word] = 1;
          keys.push(word);
        } else {
          dict[word] += 1;
        }
      });
    });

    return {
      words: keys,
      dict: dict
    };
  }

  function bow(text, vocabulary, f) {
    var dict = extractDictionary([text], f).dict,
      vector = [];

    vocabulary.words.forEach(function (word) {
      vector.push(dict[word] || 0);
    });
    return vector;
}

In [21]:
function tokenize(text) {
    return text
      .replace(/'/g, '')
      .replace(/[^A-Za-zА-Яа-яçÇğĞıİöÖşŞüÜ0-9_]/g, ' ')
      .replace(/\s\s+/g, ' ')
      .split(' ').map(function (s) {
        return s.toLowerCase();
      });
}

In [22]:
var voc_tox = extractDictionary(toxic_texsts, tokenize);   // voc = extractDictionary(X);
var voc_non_tox = extractDictionary(non_toxic_texsts, tokenize);

In [23]:
function sortProperties(obj)
{
  // convert object into array
	var sortable=[];
	for(var key in obj)
		if(obj.hasOwnProperty(key))
			sortable.push([key, obj[key]]); // each item is an array in format [key, value]
	
	// sort items by value
	sortable.sort(function(a, b)
	{
	  return b[1]-a[1]; // compare numbers
	});
	return sortable; // array in format [ [ key1, val1 ], [ key2, val2 ], ... ]
}

In [24]:
var frequent_toxic_words = sortProperties(voc_tox['dict']).map(function(p) {return p[0]});
//var frequent_non_toxic_words = sortProperties(voc_non_tox['dict']).map(function(p) {return p[0]});

In [25]:
var half_length = Math.ceil(frequent_toxic_words.length / 3);    
var frequent_toxic_words = frequent_toxic_words.splice(0,half_length);

In [26]:
var voc = extractDictionary(frequent_toxic_words.join(' '), nGram(2));   // voc = extractDictionary(X);

In [27]:
function remove_unfrequent(min, max, voc) {
    words = [];
    dict = voc['dict'];

    for (var k in dict) {
        value = dict[k];
        if(value < min || value > max) {
            delete dict[k];
        } else {
            words.push(k);
        }
    }
    
    voc.words = words;
    voc.dict = dict;
}

In [28]:
remove_unfrequent(5, 20000, voc);

In [30]:
labels.length == X.length && X_c.length == labels.length

true

In [33]:
function vectorize(text_original, voc) {
    var text = cleanString(text_original).toLowerCase();
    var vector = bow(text, voc, nGram(2))
    return vector;
}

In [34]:
var vectors = [];
for(var i = 0; i < X.length; i++) {
    var vector = vectorize(X[i], voc);
    vectors.push(vector);
}

159093

In [35]:
X_c.length == labels.length && labels.length == X.length

true

### Write the dataset out

In [36]:
fs.writeFileSync('./data/y.json', JSON.stringify(labels) , 'utf-8'); 

In [37]:
fs.writeFileSync('./data/X.json', JSON.stringify(vectors) , 'utf-8'); 

### TODO Feature engineering, kitchen sink

In [22]:
// TODO:
// try normalize if needed, skip now
// kitchen sink approach with feature engineering not just bag of words